# Technical Analysis Template: sRoC = WMA + RoC

Follow the trend the using smooth Rate of Change. The smooth function is WMA.

In [1]:
import qnt.data as qndata
import qnt.stats as qnstats
import qnt.xr_talib as qnxrtalib
import qnt.graph as qngraph
#import qnt.forward_looking as qnfl
import datetime as dt

## Data

In [2]:
data = qndata.load_data(
    tail = dt.timedelta(days=4*365),
    forward_order=True, 
    dims=("time", "field", "asset")
)

fetched chunk 1/5 1s
fetched chunk 2/5 3s
fetched chunk 3/5 4s
fetched chunk 4/5 6s
fetched chunk 5/5 7s
Data loaded 7s


## Calc output

In [3]:
SROC_POSITIVE_TREND_LEVEL = 0.0125

wma = qnxrtalib.WMA(data.sel(field='close'), 290)
sroc = qnxrtalib.ROCP(wma, 35)

is_liquid = data.sel(field="is_liquid")
weights = is_liquid.where(sroc > SROC_POSITIVE_TREND_LEVEL)

weights = weights / weights.sum("asset", skipna=True)

output = weights.fillna(0.0)

## Stats

In [4]:
stat = qnstats.calc_stat(data, output)
display(stat.to_pandas().tail())

field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2020-08-13,1.366697,0.007825,0.222443,-0.015243,-0.345248,0.493373,0.109747,1.0,774.0,0.042115,51.985576
2020-08-14,1.361325,-0.003931,0.222457,-0.019114,-0.345248,0.486796,0.108291,1.0,774.0,0.042175,51.979797
2020-08-17,1.378683,0.012751,0.222569,-0.006607,-0.345248,0.507626,0.112982,1.0,774.0,0.042198,51.978443
2020-08-18,1.379124,0.000320,0.222569,-0.006289,-0.345248,0.508160,0.113101,1.0,774.0,0.042232,51.962735
2020-08-19,1.371947,-0.005204,0.222593,-0.011460,-0.345248,0.499415,0.111166,1.0,774.0,0.042263,51.779884


## Improvement

In [5]:
stat_per_asset = qnstats.calc_stat(data, output, max_periods=40, per_asset = True)
improved_output = output.where(
    stat_per_asset.sel(field='sharpe_ratio').rolling(time=20).mean() > 0
)
stat = qnstats.calc_stat(data, improved_output, max_periods=252 * 3)
stat.to_pandas().tail()

field,equity,relative_return,volatility,underwater,max_drawdown,sharpe_ratio,mean_return,bias,instruments,avg_turnover,avg_holding_time
time,,,,,,,,,,,
2020-08-13,1.156059,0.006719,0.125964,-0.006555,-0.21593,0.393177,0.049526,1.0,634.0,0.034092,34.989310
2020-08-14,1.153334,-0.002356,0.125973,-0.008896,-0.21593,0.386600,0.048701,1.0,634.0,0.034130,34.993657
2020-08-17,1.165349,0.010417,0.126110,0.000000,-0.21593,0.414956,0.052330,1.0,634.0,0.034161,34.970347
2020-08-18,1.166406,0.000907,0.126111,0.000000,-0.21593,0.417477,0.052648,1.0,635.0,0.034182,34.961335
2020-08-19,1.162703,-0.003175,0.126126,-0.003175,-0.21593,0.408585,0.051533,1.0,635.0,0.034211,35.396290


## Checks

In [6]:
# correlation check
# your strategy should not correlate with other strategies before submission
qnstats.print_correlation(output, data)


WARNING! This strategy correlates with other strategies.
The number of systems with a larger Sharpe ratio and correlation larger than 0.8: 4
The max correlation value (with systems with a larger Sharpe ratio): 0.9938369419614559
Current sharpe ratio(3y): 0.49941489646205245



## Save output

In [7]:
qndata.write_output(output)

write output: /root/fractions.nc.gz
